In [59]:
import lightgbm as lgb
import pandas as pd # 基本ライブラリ
import numpy as np # 基本ライブラリ
import matplotlib.pyplot as plt # グラフ描画用
from sklearn.model_selection import train_test_split # データセット分割用
from sklearn.metrics import accuracy_score # モデル評価用(正答率)
from sklearn.metrics import log_loss # モデル評価用(logloss)
from sklearn.metrics import roc_auc_score # モデル評価用(auc)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


from sklearn.metrics import recall_score


# データフレームを綺麗に出力する関数
import IPython
def display(*dfs, head=True):
    for df in dfs:
        IPython.display.display(df.head() if head else df)

In [38]:
hoge = pd.read_csv("dataset/all_data_2021.csv", index_col=0)

/var/folders/s6/7jf_zlp94gg5cpb8c5lrx2j80000gn/T/ipykernel_56914/1450870400.py:1: DtypeWarning: Columns (4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  hoge = pd.read_csv("dataset/all_data_2021.csv", index_col=0)


In [39]:
hoge

,date,place_cd,race_no,SC_1,SC_2,SC_3,SC_4,SC_5,SC_6,ST_1,...,ESC_3,ESC_4,ESC_5,ESC_6,wether,air_t,wind_d,wind_v,water_t,wave_h
0,2021-05-04,21,2,1,3,4,2,5,6,0.14,...,4,2,5,6,晴,20.0,15,1.0,17.0,1.0
1,2021-05-04,21,4,1,2,3,4,5,6,0.17,...,3,4,5,6,晴,22.0,13,2.0,17.0,2.0
2,2021-05-04,21,5,1,2,3,4,5,6,0.11,...,3,4,5,6,晴,23.0,5,3.0,17.0,3.0
3,2021-05-04,21,6,1,2,3,4,5,6,0.24,...,3,4,5,6,晴,23.0,1,2.0,17.0,2.0
4,2021-05-04,21,9,1,2,3,4,5,6,0.10,...,3,4,5,6,曇り,22.0,1,3.0,17.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,2021-05-09,2,8,1,2,3,6,4,5,0.05,...,3,6,4,5,晴,27.0,16,2.0,22.0,1.0
133,2021-05-09,2,9,1,2,3,4,5,6,0.20,...,3,4,5,6,曇り,27.0,16,1.0,22.0,0.0
134,2021-05-09,2,10,1,2,3,5,4,6,0.06,...,3,4,5,6,晴,27.0,4,1.0,22.0,1.0
135,2021-05-09,2,11,1,2,3,4,5,6,0.04,...,3,4,5,6,晴,27.0,4,3.0,22.0,2.0


In [40]:
X_df = pd.read_csv("dataset/v1_features/v1.csv",index_col=0)

In [41]:
X = X_df.values

Y = hoge["tkt_1t"].values

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.20, random_state=2)

In [43]:
X_train.shape

(35838, 60)

In [44]:
y_train.shape

(35838,)

In [45]:
# モデルの学習
model = lgb.LGBMClassifier() # モデルのインスタンスの作成
model.fit(X_train, y_train) # モデルの学習

# テストデータの予測クラス (予測クラス(0 or 1 or 2)を返す)
y_pred = model.predict(X_test)
# テストデータのクラス予測確率 (各クラスの予測確率 [クラス0の予測確率,クラス1の予測確率,クラス2の予測確率] を返す)
y_pred_prob = model.predict_proba(X_test)

In [46]:
# 真値と予測値の表示
df_pred = pd.DataFrame({'target':y_test,'target_pred':y_pred})


# 真値と予測確率の表示
df_pred_prob = pd.DataFrame({'y':y_test, 'target0_prob':y_pred_prob[:,0], 'target1_prob':y_pred_prob[:,1], 'target2_prob':y_pred_prob[:,2], 'target3_prob':y_pred_prob[:,3], 'target4_prob':y_pred_prob[:,4], 'target5_prob':y_pred_prob[:,5]})

In [47]:
df_pred.to_csv("tmp.csv" , encoding='utf_8_sig')

In [48]:
np.unique(y_train)

array(['1', '2', '3', '4', '5', '6', '特払い'], dtype=object)

In [52]:
val_mat = confusion_matrix(y_test, y_pred)

In [53]:
val_mat

array([[4376,  206,  125,   89,   27,   20,   52],
       [ 886,  260,   71,   47,   14,   14,    7],
       [ 797,   90,  118,   31,   14,   17,   10],
       [ 724,   65,   37,   64,    8,    4,   15],
       [ 381,   42,   20,   10,   16,    5,    3],
       [ 204,   29,   19,    3,    3,   24,    4],
       [   9,    0,    0,    0,    0,    0,    0]])

In [55]:
accuracy_score(y_test, y_pred)


0.5421875

In [58]:
rec_score = recall_score(y_test, y_pred, average=None)
print(rec_score)

[0.89397344 0.20015396 0.1095636  0.0697928  0.03354298 0.08391608
 0.        ]


In [61]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.59      0.89      0.71      4895
           2       0.38      0.20      0.26      1299
           3       0.30      0.11      0.16      1077
           4       0.26      0.07      0.11       917
           5       0.20      0.03      0.06       477
           6       0.29      0.08      0.13       286
         特払い       0.00      0.00      0.00         9

    accuracy                           0.54      8960
   macro avg       0.29      0.20      0.20      8960
weighted avg       0.46      0.54      0.47      8960

